calculates the feature scores for each split  


In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import RandomizedLasso
import sys
import os

In [2]:
# load lb, test and CV CIDs

# load LB CIDs
with open(os.path.abspath('__file__' + "/../../../../data/CID_leaderboard.txt")) as f: 
    content = f.readlines()
lb_CIDs = list(content)  
lb_CIDs = [int(x) for x in lb_CIDs]

# load test CIDs
with open(os.path.abspath('__file__' + "/../../../../data/CID_testset.txt")) as f: 
    content = f.readlines()
test_CIDs = list(content)  
test_CIDs = [int(x) for x in test_CIDs]

In [3]:
#features = pd.read_csv('../../../data/linear_scores_and_features/features_dragon_morgan.csv')
features = pd.read_csv('features.csv')
features.head()

,CID,complexity from pubmed,MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,91305518_2,91411526_2,91541756_2,91552833_2,91563027_2,91595028_2,91614181_2,91617014_2,91617930_2,91618238_2
0,126,0.181128,0.270753,0.030587,0.262264,0.219126,0.253846,0.214989,0.216981,0.425532,...,0.000013,0.000331,0.014024,0.000296,0.021098,0.000186,0.003159,0.002299,0.000138,0.011080
1,176,0.060311,0.109331,0.025411,0.096943,0.105579,0.090940,0.107335,0.125214,0.659574,...,0.000124,0.000205,0.008391,0.000930,0.001442,0.000094,0.000607,0.001362,0.000229,0.004162
2,177,0.020039,0.067721,0.015501,0.075556,0.083688,0.078074,0.089782,0.106346,0.382979,...,0.000014,0.000092,0.000961,0.000339,0.000657,0.000008,0.000098,0.000221,0.000037,0.001932
3,180,0.051167,0.104208,0.011542,0.121231,0.131248,0.127898,0.139362,0.099485,0.269504,...,0.000124,0.000205,0.003729,0.000930,0.000641,0.000094,0.000607,0.001961,0.000229,0.001850
4,196,0.221790,0.333247,0.023779,0.306622,0.308572,0.294339,0.305729,0.138079,0.539007,...,0.001029,0.000737,0.013662,0.009383,0.001954,0.000820,0.003130,0.005600,0.002189,0.010702


In [4]:
#trainsplits = pd.read_csv('/media/gabor/H/python_from_C/final_ofaction_for_paper_2/data/cv_splits_train_big.csv',header=None)
#testsplits = pd.read_csv('/media/gabor/H/python_from_C/final_ofaction_for_paper_2/data/cv_splits_test_big.csv',header=None)

In [5]:
trainsplits = pd.read_csv('../../../data/cv_splits_train_bigger.csv',header=None)
testsplits = pd.read_csv('../../../data/cv_splits_test_bigger.csv',header=None)

In [6]:
descriptor = {}
for idx, desc in enumerate([u'INTENSITY/STRENGTH', u'VALENCE/PLEASANTNESS', u'BAKERY', 
                       u'SWEET', u'FRUIT', u'FISH', u'GARLIC', u'SPICES', u'COLD', u'SOUR', u'BURNT',
                       u'ACID', u'WARM', u'MUSKY', u'SWEATY', u'AMMONIA/URINOUS', u'DECAYED', u'WOOD',
                       u'GRASS', u'FLOWER', u'CHEMICAL']):
    descriptor[idx] = desc

In [7]:
targets = pd.read_csv('targets_for_feature_selection.csv')
targets.columns = ['CID'] + list(targets.columns.values[1:])
targets.head()

,CID,INTENSITY/STRENGTH,VALENCE/PLEASANTNESS,BAKERY,SWEET,FRUIT,FISH,GARLIC,SPICES,COLD,...,ACID,WARM,MUSKY,SWEATY,AMMONIA/URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
0,126,37.102041,50.081081,0.500000,21.959459,7.405405,0.175676,2.162162,4.554054,4.662162,...,4.094595,2.486486,7.216216,1.391892,2.554054,4.675676,0.891892,1.662162,8.094595,15.283784
1,176,8.051020,45.344828,2.275862,5.103448,1.137931,0.000000,6.448276,5.965517,4.793103,...,3.896552,5.448276,6.448276,3.551724,3.275862,4.275862,2.413793,2.482759,6.724138,7.724138
2,177,22.387755,48.418182,9.363636,19.781818,3.000000,0.763636,1.254545,2.472727,6.709091,...,3.563636,3.218182,6.218182,1.945455,2.727273,3.872727,0.727273,3.454545,4.090909,14.200000
3,196,14.530612,44.304348,1.304348,9.804348,0.913043,0.500000,3.239130,7.108696,2.152174,...,5.543478,6.695652,9.043478,7.304348,2.152174,4.217391,1.195652,1.543478,6.695652,7.847826
4,239,24.683673,51.724138,1.362069,13.500000,4.293103,1.482759,4.534483,6.189655,4.965517,...,3.241379,5.068966,6.534483,0.793103,0.931034,5.413793,3.120690,5.775862,9.396552,10.862069


In [8]:
import datetime

In [9]:
features.shape

(476, 14613)

In [13]:
import sys
sys.path.append('/Users/rgerkin/Dropbox/science/olfaction-prediction')
sys.path.append('/Users/rgerkin/Dropbox/science/olfaction-prediction/opc_python/utils')
import loading
training_leaderboard_CIDs = sorted(loading.get_CIDs('training')+loading.get_CIDs('leaderboard'))

In [17]:
from sklearn.cross_validation import ShuffleSplit
test_size = 0.17
splits = ShuffleSplit(len(training_leaderboard_CIDs),n_iter=250,test_size=test_size,random_state=0)
i = 0
for train,test in splits:
    print([training_leaderboard_CIDs[i] for i in test])
    i+=1
    if i==5:
        break

[23235, 7824, 6276, 14104, 8030, 61185, 13216, 14514, 12810, 7519, 170833, 8091, 93375, 62572, 325, 12580, 6501, 22386, 61918, 7151, 8063, 159055, 62374, 8078, 62444, 8918, 7991, 61138, 379, 6114390, 8077, 556940, 8892, 6826, 2346, 6943, 61199, 8697, 637776, 6057, 14228, 89440, 61337, 8193, 241, 176, 61204, 61177, 6213, 31249, 8093, 165675, 61192, 519539, 565690, 7463, 8857, 5363491, 61151, 6050, 263, 1001, 5541, 6997, 440967, 239, 61130, 10882, 78925, 18635]
[7127, 18635, 1549026, 7632, 10882, 6106, 2969, 311, 6213, 8078, 356, 8051, 8615, 7047, 31210, 7409, 61331, 1030, 61945, 2879, 5366244, 23642, 13436, 15606, 61177, 8174, 6276, 5363233, 7820, 11980, 8077, 454, 7761, 1068, 8205, 6137, 31260, 778574, 2345, 14257, 18827, 31252, 1060, 326, 14525, 7341, 13187, 8180, 61809, 61052, 6920, 7824, 10364, 6998, 6054, 8363, 8093, 244, 1550470, 61209, 8082, 11428, 7969, 61138, 8857, 12206, 31249, 7136, 8375, 176]
[10895, 62444, 5541, 24020, 14328, 8048, 9609, 2345, 887, 8125, 61386, 7136, 61204,

In [22]:
testsplits.head()

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
0,23235,7824,6276,14104,8030,61185,13216,14514,12810,7519,...,6050,263,1001,5541,6997,440967,239,61130,10882,78925
1,7127,18635,1549026,7632,10882,6106,2969,311,6213,8078,...,61209,8082,11428,7969,61138,8857,12206,31249,7136,8375
2,10895,62444,5541,24020,14328,8048,9609,2345,887,8125,...,638014,356,7921,93375,14296,2346,7410,7695,6287,7916
3,11980,650,89440,7601,5363388,444539,6054,62374,235414,78925,...,6998,311,7047,5368076,261,61138,7894,170833,228769,5960
4,2879,16255,15606,11527,10430,7151,5367698,641423,6943,61204,...,6584,61024,62725,61527,8186,31249,4133,519539,6998,1060


In [30]:
[int(x) if not np.isnan(x) else x for x in sorted(list(trainsplits.loc[0]))]

[126,
 177,
 196,
 240,
 243,
 244,
 261,
 311,
 326,
 356,
 454,
 460,
 650,
 660,
 679,
 807,
 875,
 887,
 957,
 994,
 1030,
 1032,
 1049,
 1060,
 1068,
 1110,
 1136,
 1183,
 2214,
 2345,
 2682,
 2758,
 2879,
 2969,
 3776,
 3893,
 4133,
 5610,
 5779,
 5780,
 5950,
 5960,
 5961,
 6054,
 6106,
 6137,
 6140,
 6184,
 6287,
 6374,
 6386,
 6448,
 6505,
 6549,
 6560,
 6569,
 6584,
 6590,
 6658,
 6753,
 6920,
 6989,
 6998,
 7047,
 7059,
 7095,
 7119,
 7122,
 7127,
 7136,
 7144,
 7147,
 7150,
 7165,
 7194,
 7288,
 7335,
 7341,
 7360,
 7361,
 7409,
 7410,
 7500,
 7583,
 7593,
 7601,
 7632,
 7635,
 7654,
 7695,
 7720,
 7731,
 7749,
 7761,
 7762,
 7765,
 7768,
 7792,
 7795,
 7799,
 7803,
 7820,
 7826,
 7894,
 7915,
 7916,
 7921,
 7937,
 7967,
 7969,
 7983,
 7997,
 8007,
 8038,
 8042,
 8048,
 8051,
 8082,
 8103,
 8118,
 8122,
 8125,
 8129,
 8137,
 8159,
 8163,
 8174,
 8175,
 8180,
 8184,
 8186,
 8205,
 8294,
 8363,
 8375,
 8452,
 8456,
 8467,
 8615,
 8635,
 8658,
 8712,
 8723,
 8785,
 8797,
 8908

In [64]:
for k in range(0,5): #set the range of splits here
    # set a cv split as holdout data.
    
    lb_CIDs = testsplits.ix[k,:].values
    features = features[~features.CID.isin(test_CIDs)] # # remove the final test data features - this seems to be reduntant after the first iteration
    #print(targets.shape,features.shape)
    
    train_targets = targets[~targets['CID'].isin(lb_CIDs)]  # remove lb_data and use the rest
    train_features = features[~features.CID.isin(lb_CIDs)] # remove lb_data and use the rest 
    train,test = list(splits)[k]
    #should_use = [training_leaderboard_CIDs[i] for i in train]
    #used = list(train_features['CID'])
    #should_use = [int(x) for x in sorted(list(trainsplits.loc[k])) if not np.isnan(x)]
    #print("   ")
    #print("k=%d" % k)
    #print("Used: n=%d" % len(used),used)
    #print("   ")
    #print("Should use: n=%d" % len(should_use),should_use)
    #print("Equal?",used == should_use)
    #print(set(used).difference(should_use))
    #print("   ")
    #print("   ")
    #print("   ")
    #print("   ")
    #print("   ")
    
    #feature selection
    #if not os.path.exists('scores/LB_scores_morgan' + str(k)): 
    #    os.makedirs('scores/LB_scores_morgan' + str(k))
    for idx in range(21):
        if k < 0 and idx < 0:  # in case the selection stops at a point, set the right numbers to continue (selection takes time)
            pass
        else:
            print(datetime.datetime.now())
            print('split ' + str(k))
            print('selection for descriptor: ' + descriptor[idx])
            sys.stdout.flush()
            Y = train_targets[descriptor[idx]]
            X = train_features.ix[:,1:]

            selector = RandomizedLasso(alpha=0.025,selection_threshold=0.001,verbose=1,n_resampling=10,
                                      random_state=12).fit(X,Y)
            
            Y = targets[descriptor[idx]]
            X = train_features.ix[:,1:]
            selector2 = RandomizedLasso(alpha=0.025,selection_threshold=0.001,verbose=1,n_resampling=10,
                                      random_state=12).fit(X,Y)


            scores = pd.DataFrame(selector.scores_,index=X.columns)
            print(selector.scores_.shape,X.columns)
            print(scores.head())
            #scores.to_csv('scores/LB_scores_morgan' + str(k) + '/scores_'+str(idx)+'.csv')
    """

   
k=0
Used: n=338 [126, 177, 196, 240, 243, 244, 261, 311, 326, 356, 454, 460, 650, 660, 679, 807, 875, 887, 957, 994, 1030, 1032, 1049, 1060, 1068, 1110, 1136, 1183, 2214, 2345, 2682, 2758, 2879, 2969, 3776, 3893, 4133, 5610, 5779, 5780, 5950, 5960, 5961, 6054, 6106, 6137, 6140, 6184, 6287, 6374, 6386, 6448, 6505, 6549, 6560, 6569, 6584, 6590, 6658, 6753, 6920, 6989, 6998, 7047, 7059, 7095, 7119, 7122, 7127, 7136, 7144, 7147, 7150, 7165, 7194, 7288, 7335, 7341, 7360, 7361, 7409, 7410, 7500, 7583, 7593, 7601, 7632, 7635, 7654, 7695, 7720, 7731, 7749, 7761, 7762, 7765, 7768, 7792, 7795, 7799, 7803, 7820, 7826, 7894, 7915, 7916, 7921, 7937, 7967, 7969, 7983, 7997, 8007, 8038, 8042, 8048, 8051, 8082, 8103, 8118, 8122, 8125, 8129, 8137, 8159, 8163, 8174, 8175, 8180, 8184, 8186, 8205, 8294, 8363, 8375, 8452, 8456, 8467, 8615, 8635, 8658, 8712, 8723, 8785, 8797, 8908, 8914, 9016, 9024, 9025, 9256, 9261, 9589, 9609, 9862, 10285, 10364, 10400, 10430, 10448, 10722, 10748, 10797, 10890, 10895,

In [66]:
len(list(splits)[k][0])

337

In [67]:
len(lb_CIDs)

69

In [60]:
X.columns.shape

(14612,)

In [61]:
train_features

,CID,complexity from pubmed,MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,91305518_2,91411526_2,91541756_2,91552833_2,91563027_2,91595028_2,91614181_2,91617014_2,91617930_2,91618238_2
0,126,0.181128,0.270753,0.030587,0.262264,0.219126,0.253846,0.214989,0.216981,0.425532,...,0.000013,0.000331,0.014024,0.000296,0.021098,0.000186,0.003159,0.002299,0.000138,0.011080
2,177,0.020039,0.067721,0.015501,0.075556,0.083688,0.078074,0.089782,0.106346,0.382979,...,0.000014,0.000092,0.000961,0.000339,0.000657,0.000008,0.000098,0.000221,0.000037,0.001932
4,196,0.221790,0.333247,0.023779,0.306622,0.308572,0.294339,0.305729,0.138079,0.539007,...,0.001029,0.000737,0.013662,0.009383,0.001954,0.000820,0.003130,0.005600,0.002189,0.010702
6,240,0.141051,0.229143,0.026007,0.240877,0.197235,0.241008,0.197436,0.214408,0.269504,...,0.000000,0.000084,0.021801,0.000134,0.008264,0.000479,0.005696,0.002347,0.000035,0.005194
8,243,0.202335,0.270753,0.030587,0.262264,0.219126,0.253846,0.214989,0.216981,0.425532,...,0.000051,0.000187,0.044910,0.000528,0.010226,0.000747,0.007957,0.003826,0.000216,0.007831
9,244,0.107782,0.234396,0.019297,0.256640,0.228315,0.262527,0.232498,0.171527,0.219858,...,0.000116,0.000189,0.025945,0.000134,0.012913,0.000906,0.009812,0.003065,0.000078,0.008116
10,261,0.048249,0.140695,0.009412,0.166906,0.178825,0.177695,0.188943,0.095197,0.205674,...,0.000217,0.000556,0.003460,0.000902,0.001355,0.000048,0.000379,0.001313,0.000325,0.006664
14,311,0.441634,0.452824,0.038815,0.355019,0.343198,0.311362,0.323340,0.180103,0.900709,...,0.000614,0.001629,0.010255,0.005364,0.007480,0.000739,0.003606,0.004504,0.001656,0.006944
17,326,0.235409,0.338604,0.016726,0.377901,0.339949,0.390425,0.346192,0.162093,0.170213,...,0.000313,0.000503,0.012979,0.001143,0.016830,0.000362,0.004182,0.003648,0.000541,0.013611
18,356,0.048638,0.250286,0.000000,0.343952,0.378270,0.385618,0.404788,0.065180,0.000000,...,0.002901,0.000751,0.002844,0.005614,0.000511,0.000420,0.001417,0.002331,0.005848,0.005102
